# Assignment 3 | Myron Bañez

## Part 1: Exploring Evictions and Code Violations in Philadelphia

### 1.1.1 Read data using `geopandas`

In [ ]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from rasterstats import zonal_stats
from rasterio.mask import mask
import hvplot.pandas
import holoviews as hv
import rasterio as rio

hv.extension('bokeh')

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
patracts = gpd.read_file("./data/PA-tracts.geojson")
patracts

### 1.1.2 Explore and trim the data  
### The data is trimmed down in order to contain information from Philadelphia.

In [ ]:
philly = ["Philadelphia County, Pennsylvania"]
philly_df =  patracts['pl'].isin(philly)
philly_demolitions = patracts.loc[philly_df]
philly_demolitions

### 1.1.3 Transform from wide to tidy format
### The data is then transformed from wide to tidy format and renaming values in the year column to understand easier.

In [ ]:
philly_demolitions_df = pd.melt(
    philly_demolitions, 
    id_vars=["GEOID", "geometry"],
    value_vars = ['e-{:02d}'.format(x) for x in range(3, 17)],
    value_name="Value", 
    var_name="Year"
)

philly_demolitions_df

In [ ]:
philly_demolitions_df_1 = philly_demolitions_df.replace(['e-03', 'e-04', 'e-05', 'e-06', 'e-07', 'e-08', 'e-09', 'e-10', 'e-11', 
                               'e-12', 'e-13', 'e-14', 'e-15', 'e-16'], 
                              [2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016])

philly_demolitions_df_1

### 1.1.4 Plot the total number of evictions per year from 2003 to 2016
### Based on the plot, the year with the greatest amount of evictions was in 2014.

In [ ]:
philly_demolitions_df_2 = philly_demolitions_df_1.groupby(['Year'])['Value'].sum()
philly_demolitions_df_2
philly_demolitions_df_2.hvplot(kind='bar', rot=90, width=900, color = '#eeba0b')

### 1.1.5 The number of evictions across Philadelphia

In [ ]:
philly_demolitions_df_3c = philly_demolitions_df_1.groupby(['GEOID'])['Value'].sum()
philly_demolitions_df_3c = philly_demolitions_df_3c.reset_index()
philly_demolitions_df_3c

In [ ]:
philly_demolitions_df_1.hvplot(c='Value', 
                     groupby = "Year",
                    frame_width=600, 
                     frame_height=500, 
                     geo=True,
                     cmap='copper_r', 
                     hover_cols=['GEOID'])

## 1.2 Code Violations in Philadelphia

### 1.2.1 Load data from 2012 to 2016

In [ ]:
violations = pd.read_csv("./data/li_violations.csv")
violations['geometry'] = gpd.points_from_xy(violations['lng'], violations['lat'])
violations = gpd.GeoDataFrame(violations, geometry='geometry', crs="EPSG:4326")
violations

### 1.2.2 Trim to specific violation types

In [ ]:
violation_types = [
    "INT-PLMBG MAINT FIXTURES-RES",
    "INT S-CEILING REPAIR/MAINT SAN",
    "PLUMBING SYSTEMS-GENERAL",
    "CO DETECTOR NEEDED",
    "INTERIOR SURFACES",
    "EXT S-ROOF REPAIR",
    "ELEC-RECEPTABLE DEFECTIVE-RES",
    "INT S-FLOOR REPAIR",
    "DRAINAGE-MAIN DRAIN REPAIR-RES",
    "DRAINAGE-DOWNSPOUT REPR/REPLC",
    "LIGHT FIXTURE DEFECTIVE-RES",
    "LICENSE-RES SFD/2FD",
    "ELECTRICAL -HAZARD",
    "VACANT PROPERTIES-GENERAL",
    "INT-PLMBG FIXTURES-RES",
]

philly_violations =  violations['violationdescription'].isin(violation_types)
phl_violations = violations.loc[philly_violations]
phl_violations = phl_violations.to_crs(epsg=3857)
phl_violations

### 1.2.3 Make a hex bin map

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), facecolor=plt.get_cmap('copper_r')(0))
x = phl_violations.geometry.x
y = phl_violations.geometry.y
ax.hexbin(x, y, gridsize=40, mincnt=1, cmap='copper_r')
philly_demolitions_df_1.to_crs(epsg=3857).plot(ax=ax, 
                                 facecolor='none', 
                                 linewidth=0.5,
                                 edgecolor='white')

ax.set_axis_off()

### 1.2.4 Spatially join data sets

In [ ]:
phl_violations_epsg = phl_violations.to_crs(epsg=3857)
philly_demolitions_df_epsg = philly_demolitions_df_1.to_crs(epsg=3857)
phl_evictions = gpd.sjoin(phl_violations_epsg, philly_demolitions_df_epsg, predicate='within', how='inner').drop(columns=['index_right'])
phl_evictions

### 1.2.5 Calculate the number of violations by type per census tract

In [ ]:
phl_evictions_1 = phl_evictions.groupby(['GEOID', 'violationdescription'])['Value'].size().unstack(fill_value=0).stack().reset_index(name='n')
phl_evictions_1

### 1.2.6 Merge with census tracts geometries

In [ ]:
philly_demolitions_df_3 = gpd.GeoDataFrame(philly_demolitions_df_1, geometry='geometry', crs="EPSG:4326")
philly_demolitions_df_3
phl_evictions_2 = philly_demolitions_df_3.merge(phl_evictions_1, on='GEOID')
phl_evictions_2

### 1.2.7 Interactive choropleths for each violation type

In [ ]:
phl_evictions_2.hvplot(c='n', 
                     groupby = 'violationdescription',
                    frame_width=600, 
                     frame_height=500, 
                     geo=True,
                       dynamic=False,
                     cmap='copper_r', 
                     hover_cols=['GEOID'])

## 1.3. A side-by-side comparison

In [ ]:
year = [2016]
philly_demolitions_df_year =  philly_demolitions_df_1['Year'].isin(year)
philly_demolitions_2016 = philly_demolitions_df_1.loc[philly_demolitions_df_year]
philly_demolitions_2016

year2016 = philly_demolitions_2016.hvplot(c='Value', 
                    frame_width=600, 
                     frame_height=500, 
                     geo=True,
                     cmap='copper_r', 
                     hover_cols=['GEOID'])

year2016

In [ ]:
viotype = ["ELECTRICAL -HAZARD"]
phl_evictions_viotype =  phl_evictions_2['violationdescription'].isin(viotype)
phl_evictions_electric = phl_evictions_2.loc[phl_evictions_viotype]
phl_evictions_electric

electrical_hazard = phl_evictions_electric.hvplot(c='Value',
                              groupby = "Year",
                    frame_width=600, 
                     frame_height=500, 
                     geo=True,
                     cmap='copper_r', 
                     hover_cols=['GEOID'])

In [ ]:
combined = year2016 + electrical_hazard
combined

## Part 2: Exploring the NDVI in Philadelphia

## 2.1 Comparing the NDVI in the city and the suburbs

### 2.1.1 Load Landsat data for Philadelphia

In [ ]:
landsat = rio.open("./data/landsat8_philly.tif")
landsat
data = landsat.read(1)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
img = ax.imshow(
    data,
    norm=mcolors.LogNorm(),  # Use a log colorbar scale
    extent=[  # Set the extent of the images
        landsat.bounds.left,
        landsat.bounds.right,
        landsat.bounds.bottom,
        landsat.bounds.top,
    ],
    cmap="copper",
)
plt.colorbar(img)

### 2.1.2 Separating the city from the suburbs

### City polygon

In [ ]:
city_limits = gpd.read_file("./data/City_Limits.geojson")
city_limits = city_limits.to_crs(landsat.crs.data['init'])
city_limits

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
landsat_extent = [
    landsat.bounds.left,
    landsat.bounds.right,
    landsat.bounds.bottom,
    landsat.bounds.top,
]
img = ax.imshow(data, 
                norm=mcolors.LogNorm(), #NEW
                extent=landsat_extent,
               cmap = "copper")  #NEW
city_limits.plot(ax=ax, facecolor="none", edgecolor="white")
plt.colorbar(img)
ax.set_axis_off()

### Suburb polygon

In [ ]:
suburbs = city_limits.envelope
suburbs = suburbs.difference(city_limits.geometry)
suburbs = suburbs.to_crs(landsat.crs.data['init'])
suburbs

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
landsat_extent = [
    landsat.bounds.left,
    landsat.bounds.right,
    landsat.bounds.bottom,
    landsat.bounds.top,
]
img = ax.imshow(data, 
                norm=mcolors.LogNorm(), #NEW
                extent=landsat_extent,
               cmap="copper")  #NEW
suburbs.plot(ax=ax, facecolor="none", edgecolor="white")
plt.colorbar(img)
ax.set_axis_off()

### 2.1.3 Mask and calculate the NDVI for the city and the suburbs

### Creating the mask for the city.

In [ ]:
masked, mask_transform = mask(
    dataset=landsat,
    shapes=city_limits.geometry,
    crop=True,  # remove pixels not within boundary
    all_touched=True,  # get all pixels that touch the boudnary
    filled=False,  # do not fill cropped pixels with a default value
)

# Initialize
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the first band
ax.imshow(masked[0], cmap="copper", extent=landsat_extent)

# Format and add the city limits
city_limits.boundary.plot(ax=ax, color="gray", linewidth=4)
ax.set_axis_off()

### Calculating the NDVI for the city.

In [ ]:
masked.shape

red = masked[3]
nir = masked[4]

red.mask

def calculate_NDVI(nir, red):
    """
    Calculate the NDVI from the NIR and red landsat bands
    """
    
    # Convert to floats
    nir = nir.astype(float)
    red = red.astype(float)
    
    # Get valid entries
    check = np.logical_and( red.mask == False, nir.mask == False )
    
    # Where the check is True, return the NDVI, else return NaN
    ndvi = np.where(check,  (nir - red ) / ( nir + red ), np.nan )
    return ndvi 
NDVI = calculate_NDVI(nir, red)
fig, ax = plt.subplots(figsize=(10,10))

# Plot NDVI
img = ax.imshow(NDVI, extent=landsat_extent, cmap="copper")

# Format and plot city limits
city_limits.plot(ax=ax, edgecolor='gray', facecolor='none', linewidth=4)
plt.colorbar(img)
ax.set_axis_off()
ax.set_title("NDVI in Philadelphia", fontsize=18);

### Creating the mask for the suburbs.

In [ ]:
masked1, mask_transform = mask(
    dataset=landsat,
    shapes=suburbs,
    crop=True,  # remove pixels not within boundary
    all_touched=True,  # get all pixels that touch the boudnary
    filled=False,  # do not fill cropped pixels with a default value
)

# Initialize
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the first band
ax.imshow(masked1[0], cmap="copper", extent=landsat_extent)

# Format and add the city limits
suburbs.boundary.plot(ax=ax, color="gray", linewidth=4)
ax.set_axis_off()

### Calculating the NDVI for the suburbs.

In [ ]:
masked1.shape

red = masked1[3]
nir = masked1[4]

red.mask

def calculate_NDVI(nir, red):
    """
    Calculate the NDVI from the NIR and red landsat bands
    """
    
    # Convert to floats
    nir = nir.astype(float)
    red = red.astype(float)
    
    # Get valid entries
    check = np.logical_and( red.mask == False, nir.mask == False )
    
    # Where the check is True, return the NDVI, else return NaN
    ndvi = np.where(check,  (nir - red ) / ( nir + red ), np.nan )
    return ndvi 
NDVI_1 = calculate_NDVI(nir, red)
fig, ax = plt.subplots(figsize=(10,10))

# Plot NDVI
img = ax.imshow(NDVI_1, extent=landsat_extent, cmap='copper')

# Format and plot city limits
city_limits.plot(ax=ax, edgecolor='gray', facecolor='none', linewidth=4)
plt.colorbar(img)
ax.set_axis_off()
ax.set_title("NDVI in Philadelphia Suburbs", fontsize=18);

### 2.1.4 Calculate the median NDVI within the city and within the suburbs

In [ ]:
stats = zonal_stats(city_limits, NDVI, affine=landsat.transform, stats=['median'], nodata=np.nan)
city_limits['median_NDVI'] = [s['median'] for s in stats] 

stats1 = zonal_stats(suburbs, NDVI, affine=landsat.transform, stats=['median'], nodata=np.nan)
suburbs['median_NDVI'] = [s['median'] for s in stats1] 

### The Philadelphia suburbs have a higher NDVI at .29 compared to the city proper at .20.

In [ ]:
city_limits['median_NDVI'] #0.20
suburbs['median_NDVI'] #0.29

## 2.2 Calculating the NDVI for Philadelphia's street treets

### 2.2.1 Load the street tree data

In [ ]:
philly_trees = gpd.read_file("./data/ppr_tree_canopy_points_2015.geojson")
philly_trees.crs
landsat.crs
landsat.crs.data['init']
philly_trees = philly_trees.to_crs(landsat.crs.data['init'])

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
img = ax.imshow(NDVI, extent=landsat_extent, cmap='copper')
philly_trees.plot(ax=ax, edgecolor='#eeba0b', facecolor='none', linewidth=2)
city_limits.plot(ax=ax, edgecolor='gray', facecolor='none', linewidth=4)
plt.colorbar(img)
ax.set_axis_off()
ax.set_title("NDVI vs. Trees in Philadelphia", fontsize=18);

### 2.2.2 Calculate the NDVI values at the locations of the street trees

In [ ]:
treestats = zonal_stats(philly_trees, NDVI, affine=landsat.transform, stats=['median'], nodata=np.nan)
treestats

philly_trees['median_NDVI'] = [s['median'] for s in treestats] 
philly_trees

### 2.2.3 Plotting the results

In [ ]:
final_hist = fig, ax = plt.subplots(figsize=(8,6))
ax.hist(philly_trees['median_NDVI'], bins='auto', color="#eeba0b")
ax.axvline(x=0, c='k', lw=2)
ax.set_xlabel("Median NDVI", fontsize=18)
ax.set_ylabel("Number of Trees", fontsize=18);


In [ ]:
final_plot = fig, ax = plt.subplots(figsize=(10,10))
city_limits.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=4)
philly_trees.plot(column='median_NDVI', legend=True, ax=ax, cmap='copper_r')
ax.set_axis_off()